<a href="https://colab.research.google.com/github/guiraposo/GRpy/blob/main/TOV_in_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sympy

In [20]:
import sympy as sp

In [40]:
t, theta, phi= sp.symbols('t theta phi ')
M, r = sp.symbols('M r', positive=True)

A = sp.Function('A')(r)
B = sp.Function('B')(r)
g = sp.Matrix([[- A**2, 0, 0, 0],
               [0, B**2, 0, 0],
               [0, 0, r**2, 0],
               [0, 0, 0, r**2 * sp.sin(theta)**2]])

In [101]:
def christoffel_symbols(metric, coords):
    dims = metric.shape[0]
    christoffel = [[[0 for _ in range(dims)] for _ in range(dims)] for _ in range(dims)]
    inv_metric = metric.inv()
    for k in range(dims):
        for i in range(dims):
            for j in range(dims):
                sum_term = 0
                for l in range(dims):
                    partial_1 = sp.simplify(sp.diff(metric[j, l], coords[i]))
                    partial_2 = sp.simplify(sp.diff(metric[i, l], coords[j]))
                    partial_3 = sp.simplify(sp.diff(metric[i, j], coords[l]))
                    sum_term += inv_metric[k, l] * (partial_1 + partial_2 - partial_3)
                christoffel[k][i][j] = sp.simplify(sum_term / 2)
    return christoffel
def riemann_tensor(christoffel, coords):
    dims = len(coords)
    R = [[[sp.zeros(dims, dims) for _ in range(dims)] for _ in range(dims)] for _ in range(dims)]
    for rho in range(dims):
        for sigma in range(dims):
            for mu in range(dims):
                for nu in range(dims):
                    term1 = sp.simplify(sp.diff(christoffel[rho][nu][sigma], coords[mu]))
                    term2 = sp.simplify(sp.diff(christoffel[rho][mu][sigma], coords[nu]))
                    term3 = sp.simplify(sum(christoffel[rho][mu][lambda1] * christoffel[lambda1][nu][sigma] for lambda1 in range(dims)))
                    term4 = sp.simplify(sum(christoffel[rho][nu][lambda1] * christoffel[lambda1][mu][sigma] for lambda1 in range(dims)))
                    R[rho][sigma][mu][nu] = sp.simplify(term1 - term2 + term3 - term4)
    return R
def ricci_tensor(riemann, coords):
    dims = len(coords)
    ricci = sp.zeros(dims, dims)
    for mu in range(dims):
        for nu in range(dims):
            ricci[mu, nu] = sp.simplify(sum(riemann[lambda1][mu][lambda1][nu] for lambda1 in range(dims)))
    return ricci
def ricci_scalar(ricci, metric):
    inv_metric = metric.inv()
    dims = metric.shape[0]
    scalar = 0
    for mu in range(dims):
        for nu in range(dims):
            scalar += sp.simplify(inv_metric[mu, nu] * ricci[mu, nu])
    return sp.simplify(scalar)


In [102]:
coords = [t, r, theta, phi]  # Assuming these are your coordinates
christoffel_symbols_g = christoffel_symbols(g, coords)
riemann_tensor_g = riemann_tensor(christoffel_symbols_g, coords)
ricci_tensor_g = ricci_tensor(riemann_tensor_g, coords)
ricci_scalar_g = ricci_scalar(ricci_tensor_g, g)

In [103]:
A_test = sp.sqrt(1 - 2*M / r)
B_test = sp.sqrt(1 - 2*M / r)**(-1)
christoffel_symbols_substituted = [[[
    sp.simplify(symbol.subs({A: A_test, B: B_test}))
    for symbol in row]
    for row in matrix]
    for matrix in christoffel_symbols_g]


In [107]:
sp.simplify(sp.simplify(ricci_scalar_g.subs({A: A_test, B: B_test})))

0